# Project Real-time Finance

_(ENG)_
**Objective:** To analyze stocks data from IBOVESPA and create a time series model that could predict in short or mid-term the stocks' price based on historical data, such as:

1. Stocks' history.
2. Quarterly results (Q4).
3. News.

_(PT-BR)_
**Objetivo:** Analisar os dados de ativos da IBOVESPA e criar um modelo de times series que possa prever a curto ou médio prazo o preço das ações com base em dados históricos da empresa, tais como:

1. Valores dos ativos historicamente.
2. Resultados trimestrais (TRI).
3. Notícias.

## 1.1. Imports

In [1]:
# Data Wrangling
import pandas as pd
import numpy as np
import os
import time

# datasets of finance
import yfinance as yf

# webscrapping and manipulating endpoints
import requests
from bs4 import BeautifulSoup as BS

# Data visualization
import dash
from matplotlib import pyplot as plt
import seaborn

## 1.2. Loading and scraping Datasets

### 1.2.1. Loading IPCA (consumption interest rate)

In [2]:
##############################################################################################################################
# Webscraping done in the SINDRA website 'https://sidra.ibge.gov.br/tabela/1737' from 
# 'https://dados.gov.br/dados/conjuntos-dados', 'https://dados.gov.br/dados/conjuntos-dados/ia-indice-nacional-de-precos-ao-consumidor-amplo-ipca'
##############################################################################################################################

head = {'user-agent':'Mozilla/5.0'}
answer = requests.get('https://sidra.ibge.gov.br/geratabela?format=br.csv&name=tabela1737.csv&terr=NCS&rank=-&query=t/1737/n1/all/v/2266/p/all/d/v2266%2013/l/v,p,t&measurecol=true', 
                      headers = head)
answer.text

'ï»¿"Tabela 1737 - IPCA - SÃ©rie histÃ³rica com nÃºmero-Ã\xadndice, variaÃ§Ã£o mensal e variaÃ§Ãµes acumuladas em 3 meses, em 6 meses, no ano e em 12 meses (a partir de dezembro/1979)"\r\n"VariÃ¡vel - IPCA - NÃºmero-Ã\xadndice (base: dezembro de 1993 = 100)"\r\n"NÃ\xadvel";"CÃ³d.";"";"MÃªs"\r\n"NÃ\xadvel";"CÃ³d.";"";"dezembro 1979";;"janeiro 1980";;"fevereiro 1980";;"marÃ§o 1980";;"abril 1980";;"maio 1980";;"junho 1980";;"julho 1980";;"agosto 1980";;"setembro 1980";;"outubro 1980";;"novembro 1980";;"dezembro 1980";;"janeiro 1981";;"fevereiro 1981";;"marÃ§o 1981";;"abril 1981";;"maio 1981";;"junho 1981";;"julho 1981";;"agosto 1981";;"setembro 1981";;"outubro 1981";;"novembro 1981";;"dezembro 1981";;"janeiro 1982";;"fevereiro 1982";;"marÃ§o 1982";;"abril 1982";;"maio 1982";;"junho 1982";;"julho 1982";;"agosto 1982";;"setembro 1982";;"outubro 1982";;"novembro 1982";;"dezembro 1982";;"janeiro 1983";;"fevereiro 1983";;"marÃ§o 1983";;"abril 1983";;"maio 1983";;"junho 1983";;"julho 1983";;"ag

In [3]:
soup = BS(answer.content, 'html.parser')
soup = soup.text
soup

'"Tabela 1737 - IPCA - Série histórica com número-índice, variação mensal e variações acumuladas em 3 meses, em 6 meses, no ano e em 12 meses (a partir de dezembro/1979)"\r\n"Variável - IPCA - Número-índice (base: dezembro de 1993 = 100)"\r\n"Nível";"Cód.";"";"Mês"\r\n"Nível";"Cód.";"";"dezembro 1979";;"janeiro 1980";;"fevereiro 1980";;"março 1980";;"abril 1980";;"maio 1980";;"junho 1980";;"julho 1980";;"agosto 1980";;"setembro 1980";;"outubro 1980";;"novembro 1980";;"dezembro 1980";;"janeiro 1981";;"fevereiro 1981";;"março 1981";;"abril 1981";;"maio 1981";;"junho 1981";;"julho 1981";;"agosto 1981";;"setembro 1981";;"outubro 1981";;"novembro 1981";;"dezembro 1981";;"janeiro 1982";;"fevereiro 1982";;"março 1982";;"abril 1982";;"maio 1982";;"junho 1982";;"julho 1982";;"agosto 1982";;"setembro 1982";;"outubro 1982";;"novembro 1982";;"dezembro 1982";;"janeiro 1983";;"fevereiro 1983";;"março 1983";;"abril 1983";;"maio 1983";;"junho 1983";;"julho 1983";;"agosto 1983";;"setembro 1983";;"outub

In [4]:
pd.DataFrame(soup.split(';'))[6:-8]

,0
6,"""dezembro 1979"""
7,
8,"""janeiro 1980"""
9,
10,"""fevereiro 1980"""
...,...
2081,"""6508,4000000000000"""
2082,"""Número-índice"""
2083,"""6563,0700000000000"""
2084,"""Número-índice"""


In [5]:
# help(BS)
# help(yfinance)

### 1.2.2. Loading stocks dataset

In [6]:
tic = time.time()

# Defining the stock ticker (symbol)
tickers = ["ITSA4.SA"]

# obtaining the data
stocks = {}
for i in tickers:
    stock = yf.Ticker(tickers[0])
    stocks[i] = stock.history(period="5y", interval="1d")

# # Print the stocks data
# for i, data in stocks.items:
#     print(f"{i}")
#     print(data.tail())

toc = time.time()
time_loading02 = toc - tic
print(f'Time loading stocks dataset: {time_loading02}')

stocks['ITSA4.SA']

Time loading stocks dataset: 0.8619523048400879


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-04-30 00:00:00-03:00,8.322322,8.352673,8.213057,8.261619,14354109,0.0,0.0
2018-05-02 00:00:00-03:00,8.206990,8.206990,7.739581,7.769931,50261107,0.0,0.0
2018-05-03 00:00:00-03:00,7.800282,7.854914,7.545331,7.557471,46139605,0.0,0.0
2018-05-04 00:00:00-03:00,7.551402,7.587824,7.436068,7.436068,28024816,0.0,0.0
2018-05-07 00:00:00-03:00,7.472488,7.502839,7.411785,7.448207,17477633,0.0,0.0
...,...,...,...,...,...,...,...
2023-04-24 00:00:00-03:00,8.510000,8.670000,8.480000,8.570000,25977100,0.0,0.0
2023-04-25 00:00:00-03:00,8.530000,8.600000,8.520000,8.590000,27183900,0.0,0.0
2023-04-26 00:00:00-03:00,8.600000,8.600000,8.450000,8.480000,25655800,0.0,0.0
